In [1]:
# Hidden cell for imports
import pychor
import galois
import numpy as np

p1 = pychor.Party('p1')
p2 = pychor.Party('p2')

# Oblivious Transfer

Oblivious Transfer (OT) is a fundamental primitive used in many MPC protocols. As we'll see at the end of this chapter, it can be used to build a protocol for generating multiplication triples. It is also used as part of the online portion of other MPC protocols, as we'll see in later chapters.

````{admonition} Further reading: Oblivious Transfer
:class: seealso

For more information on Oblivious Transfer, see **Sections 2.4 and 3.7 of [Pragmatic MPC](https://securecomputation.org/)**.
````

## The Oblivious Transfer Functionality

OT is a protocol between two parties: the *Sender* and the *Receiver*. The Sender provides two inputs, and the Receiver selects one of them. At the end of the protocol, the Receiver gets *only* one of the Sender's inputs (the selected one), and the Sender *does not know which input was selected* (this is the "oblivious" part - the Sender is oblivious to the selection decision). In fact, the Sender does not receive any output from OT. The ideal functionality for this version of OT, which is called 1-out-of-2 OT because the sender provides two inputs, is defined as follows:

````{admonition} Ideal functionality: 1-out-of-2 Oblivious Transfer
:class: tip

1. The Sender sends the secret inputs $x_0$ and $x_1$ to $\mathcal{F}_\text{OT}$
2. The Receiver sends the selection bit $s$ to $\mathcal{F}_\text{OT}$
3. If $s = 0$, $\mathcal{F}_\text{OT}$ sends $x_0$ to the Receiver; if $s = 1$, $\mathcal{F}_\text{OT}$ sends $x_1$ to the Receiver
````

We can implement the functionality as follows. We'll use integers for the inputs and the selection.

In [2]:
sender = pychor.Party('sender')
receiver = pychor.Party('receiver')
FOT = pychor.Party('FOT')

@pychor.local_function
def select(inputs, selection):
    return inputs[selection]

def functionality_ot(sender, receiver, inputs, selection):
    inputs.send(sender, FOT)
    selection.send(receiver, FOT)
    selected_input = select(inputs, selection)
    selected_input.send(FOT, receiver)
    return selected_input

In [3]:
with pychor.LocalBackend():
    inputs = sender.constant([42, 67])

    selection = receiver.constant(0)
    selected_input = functionality_ot(sender, receiver, inputs, selection)
    print('OT Result (s=0):', selected_input)

    selection = receiver.constant(1)
    selected_input = functionality_ot(sender, receiver, inputs, selection)
    print('OT Result (s=1):', selected_input)

OT Result (s=0): 42@{FOT, receiver}
OT Result (s=1): 67@{FOT, receiver}


## A Protocol for Oblivious Transfer

In this section we'll define a simple protocol for OT that realizes the functionality in the semi-honest setting. Our approach follows Section 3.7 of [Pragmatic MPC](https://securecomputation.org/) and uses [public-key cryptography](https://en.wikipedia.org/wiki/Public-key_cryptography) as a fundamental building block. The asymmetry between Sender and Receiver in OT is very different from what we've seen in prior chapters, and it turns out that this asymmetry creates a deep link between OT and public-key cryptography (see Gertner et al. {cite}`gertner2000relationship`) for details).

We'll use the [PyNaCl](https://pypi.org/project/PyNaCl/) library for public-key cryptography. This library is a wrapper around [libsodium](https://github.com/jedisct1/libsodium), a high-performance cryptography library. The [documentation for public-key cryptography in PyNaCl can be found here](https://pynacl.readthedocs.io/en/latest/public/). Here's a simple example that generates a key pair, encrypts a value, and then decrypts the value. When using the PyNaCl library, everything is bytes (input values, keys, encrypted values, etc) - so we will often need to encode the values we want to encrypt into bytestrings before doing so.

In [4]:
from nacl.public import PrivateKey, PublicKey, SealedBox

# Generate the key pair (secret key and public key)
key_pair = PrivateKey.generate()
print('Key pair:', key_pair)

# Encrypt a value using the public key from the key pair
pk = key_pair.public_key
encrypted_value = SealedBox(pk).encrypt(b'this is a test')
print('Encrypted value:', encrypted_value)

# Decrypt the value using the secret key from the key pair
decrypted_value = SealedBox(key_pair).decrypt(encrypted_value)
print('Decrypted value:', decrypted_value)

Key pair: b"\x15VGR?\x1f\xb1\xcf\x8em\x8b\xfd\xed\x92\x95\xf6B\x81a\x0c^\x1e.\x87\x8c'>\x1a\xd4f\x8c\x8b"
Encrypted value: b'3\xa9p\xb1\xb2_,8\x17o\x98\xb7!\xd9\xc0\x84\x9e\xf38\xd6F\x1f\xc8\xec.\xe3 \x7f%\x1b.Pd\xa5\x1e\xe9\xd4\xecc\xab\xe9\xd6Zo\x0eN\x8d\xd5*LcN!\x15\x03\xff\xa0\xd7\x91\x19\xd1\xd5'
Decrypted value: b'this is a test'


In our OT protocol, the basic idea is for the Receiver to send two public keys to the Sender, and for the Sender to encrypt *both* secret inputs (one input per public key) and send the encrypted inputs back to the Receiver. However, the two public keys are special: one is generated as part of a key pair (so the Receiver knows the corresponding secret key), and the other is *totally random* (so no secret key exists, and *the Receiver cannot decrypt any message encrypted with it*). Which one is "real" and which one is random depends on the selection bit. This approach satisfies the requirements of the ideal functionality: the Receiver gets only the selected input (because they can't decrypt the other one), and the Sender gets nothing (because the two public keys both look random, without knowledge of how they were generated).

Note that this protocol is *very* insecure in the presence of malicious adversaries. For example, a malicious Receiver could send two "real" public keys to the Sender, and then trivially decrypt both inputs.

````{admonition} Protocol: 1-out-of-2 Oblivious Transfer
:class: note

Setup:
- Sender has secret inputs $x_0$ and $x_1$
- Receiver has selection bit $s$

The parties Sender and Receiver follow the following steps:
1. The Receiver generates a key pair $sk$, $pk$, and samples a random public key, $pk'$, from the public key space. If $s = 0$, the Receiver sends the pair $(pk, pk')$ to the Sender; if $s = 1$, the receiver sends the pair $(pk', pk)$ to the Sender.
2. The Sender receives $(pk_0, pk_1)$ and sends back to the Receiver two encryptions $e_0 = \text{Enc}_{pk_0}(x_0), e_1 = \text{Enc}_{pk_1}(x_1)$.
3. The Receiver receives $e_0, e_1$ and decrypts the ciphertext $e_s$ using $sk$. The Receiver is unable to decrypt the second ciphertext as it does not have the corresponding secret key.
````

The implementation follows the same steps, but uses the PyNaCl API to perform the key generation, encryption, and decryption operations. Since PyNaCl only supports encrypting bytes, the protocol converts the secret inputs to bytestrings (the protocol expects the inputs to be integers, and returns an integer).

Our implementation also generalizes the idea to 1-out-of-$n$ OT, where $n$ is any positive integer. To do this, the Receiver provides a selection index as an integer between 0 and $n-1$ (rather than just a bit), the Receiver generates $n-1$ fake public keys (rather than 1), the Sender sends $n$ encrypted values to the Receiver, and the Receiver is able to decrypt only one of them. Common variants of OT are 1-out-of-2 (as described above) and 1-out-of-4 (as we'll use in the next section).

In [5]:
from nacl.utils import random

# Protocol for 1-out-of-n OT
def protocol_ot(sender, receiver, inputs, selection, n):
    # Function for the Receiver to generate keys
    @pychor.local_function
    def gen_keys(selection, n):
        # Generate a single real key pair key = (sk, pk)
        key = PrivateKey.generate()
        public_keys = [PublicKey(random(PublicKey.SIZE)) for _ in range(n)]
        public_keys[selection] = key.public_key
        return key, public_keys

    # Function for the Sender to encrypt the secret inputs
    @pychor.local_function
    def encrypt_inputs(pub_keys, inputs):
        # Encode the inputs as bytes
        length = max([(int(x).bit_length() + 7) // 8 for x in inputs])
        inputs_bytes = [int(x).to_bytes(length, 'little') for x in inputs]
    
        # Encrypt the inputs
        encrypted_inputs = [SealedBox(pk).encrypt(x) for pk, x in \
                            zip(pub_keys, inputs_bytes)]
        return encrypted_inputs

    # Function for the Receiver to decrypt the result
    @pychor.local_function
    def decrypt_result(selection, key, encrypted_inputs):
        # Select the correct input
        selected_input = encrypted_inputs[selection]
        # Decrypt it and convert it from bytes to int
        plaintext = SealedBox(key).decrypt(selected_input)
        return int.from_bytes(plaintext, 'little')

    # Step 1: Generate keys and send to Sender
    sk, pub_keys = gen_keys(selection, n).untup(2)
    pub_keys.send(receiver, sender)

    # Step 2: Encrypt inputs and send to Receiver
    encrypted_inputs = encrypt_inputs(pub_keys, inputs)
    encrypted_inputs.send(sender, receiver)

    # Step 3: Decrypt result
    result = decrypt_result(selection, sk, encrypted_inputs)

    return result

In [6]:
with pychor.LocalBackend():
    inputs = sender.constant([42, 67])

    selection = receiver.constant(0)
    selected_input = protocol_ot(sender, receiver, inputs, selection, 2)
    print('OT Result (s=0):', selected_input)

    selection = receiver.constant(1)
    selected_input = protocol_ot(sender, receiver, inputs, selection, 2)
    print('OT Result (s=1):', selected_input)

OT Result (s=0): 42@{receiver}
OT Result (s=1): 67@{receiver}


## Application: Generating Binary Multiplication Triples using OT

One important use of OT is to generate multiplication triples. We previously introduced the ideal functionality for generating multiplication triples, but we didn't implement a protocol realizing it because we didn't have the necessary tools. With OT, now we do!

We'll start with the case of generating *binary* multiplication triples: the case where all secret values and secret shares are in $GF(2)$ (i.e. either 0 or 1). Since $GF(2)$ is a finite field, the protocols we built earlier that use multiplication triples will work just fine in this case.

Recall that our goal is to generate secret-shared $a$, $b$, and $c$ such that $a*b = c$. Specifically, we want to end up in the following situation:
- P1 knows $a_1$, $b_1$, $c_1$
- P2 knows $a_2$, $b_2$, $c_2$
- $(a_1+a_2)(b_1 + b_2) = c_1 + c_2$

To get there, we'll use this basic idea:
1. P1 picks $a_1$, $b_1$, $c_1$ randomly
2. P2 picks $a_2$, $b_2$ randomly
3. P1 and P2 run OT, with P1 as sender and P2 as receiver, so that P2 ends up with $c_2$

What should the secret inputs and selection bit for OT be? We actually need to use *two* selection bits: the values of P2's shares ($a_2$ and $b_2$). For the inputs, since we're working in $GF(2)$, P1 can build the a big truth table to list the possible values of $c_2$, and use the truth table to generate the set of secret inputs. An excerpt of the complete truth table for valid multiplication triples looks like this:

| $a_1$ | $b_1$ | $c_1$ | $a_2$ | $b_2$ | $c_2$ |
|-------|-------|-------|-------|-------|-------|
| 0     | 0     | 0     | 0     | 0     | 0     |
| 0     | 0     | 0     | 0     | 1     | 0     |
| 0     | 0     | 0     | 1     | 0     | 0     |
| 0     | 0     | 0     | 1     | 1     | 1     |
...

The complete table is large, so we don't show it here. But to generate multiplication triples, we won't actually have to build the whole thing. When P1 builds the truth table to use in OT, P1 actually knows many of the values of the secret shares listed in it, and so most of the rows in the table can be eliminated. The only values P1 *doesn't* know are $a_2$ and $b_2$, and there are only four possible combinations of those values. So, we can write a function that builds the truth table for $a_2$ and $b_2$, given the values of the other shares:

In [7]:
GF_2 = galois.GF(2)

@pychor.local_function
def truth_table(a1, b1, c1):
    possible_c2s = []
    # Consider all possibilities for a2 and b2
    for a2 in GF_2([0, 1]):
        for b2 in GF_2([0, 1]):
            # Compute the share c2 in terms of the others
            c2 = (a1+a2) * (b1+b2) - c1
            possible_c2s.append(c2)
    return possible_c2s

truth_table(GF_2(0), GF_2(0), GF_2(0))

[GF(0, order=2), GF(0, order=2), GF(0, order=2), GF(1, order=2)]

Note that these values are the first four values of $c_2$ in the truth table excerpt above.

We can use 1-out-of-4 OT to deliver the value $c_2$ to P2 without P1 knowing what it is. We'll use the truth table generated by `truth_table` as the secret inputs, and the values of $a_2$ and $b_2$ as selection bits to build the selection index.

The complete protocol for generating a binary multiplication triple can be defined as follows:

````{admonition} Protocol: Generate Binary Multiplication Triple
:class: note

The parties P1 and P2 follow the following steps:
1. P1 picks $a_1$, $b_1$, and $c_1 \in GF(2)$ randomly
2. P2 picks $a_2$, $b_2 \in GF(2)$ randomly
3. P1 generates the truth table based on its shares by running `truth_table`
4. P1 and P2 run 1-out-of-4 OT, with P1 as sender and P2 as receiver. P1 submits the truth table as the set of secret inputs, and P2 submits the value $2*a_2 + b_2$ as the selection index. P2 receives the output of OT, which is the value of $c_2$.

At the end of the protocol, P1 and P2 hold secret shares of $a$, $b$, and $c$, such that $a$ and $b$ are random and unknown to either party, and $a*b = c$.
````

This protocol realizes the ideal functionality described in chapter 4, in the presence of a semi-honest adversary. We can replace uses of that functionality in applications that perform multiplication using triples to avoid the need for a trusted party to generate the triples; this is how most actual deployments of triple-based protocols work.

In [8]:
def protocol_gen_binary_mult_triple():
    # Function for computing the selection index
    @pychor.local_function
    def compute_selection(a2, b2):
        return int(a2)*2 + int(b2)

    # Step 1: Pick P1's shares
    a1 = p1.constant(GF_2.Random())
    b1 = p1.constant(GF_2.Random())
    c1 = p1.constant(GF_2.Random())

    # Step 2: Pick P2's shares
    a2 = p2.constant(GF_2.Random())
    b2 = p2.constant(GF_2.Random())

    # Step 3: Generate truth table
    table = truth_table(a1, b1, c1)

    # Step 4: Run 1-out-of-4 OT to deliver c2 to P2
    selection = compute_selection(a2, b2)
    c2_val = protocol_ot(sender=p1, receiver=p2, 
                         inputs=table, 
                         selection=selection, 
                         n=4)
    c2 = pychor.locally(GF_2, c2_val)
    return (a1, a2), (b1, b2), (c1, c2)

In [9]:
# Protocol for checking a triple by reconstruction
def test_triple(triple):
    (a1, a2), (b1, b2), (c1, c2) = triple
    a2.send(p2, p1)
    b2.send(p2, p1)
    c2.send(p2, p1)
    ab = (a1+a2) * (b1+b2)
    c = c1+c2
    print(f'a*b: {ab}, c: {c}; are they equal? {ab == c}')

with pychor.LocalBackend():
    for _ in range(5):
        # Generate a triple
        triple = protocol_gen_binary_mult_triple()
        # Verify it's correct
        test_triple(triple)

a*b: 0@{p1}, c: 0@{p1}; are they equal? True
a*b: 1@{p1}, c: 1@{p1}; are they equal? True
a*b: 1@{p1}, c: 1@{p1}; are they equal? True
a*b: 1@{p1}, c: 1@{p1}; are they equal? True
a*b: 1@{p1}, c: 1@{p1}; are they equal? True


## Application: Generating Arithmetic Multiplication Triples using OT

We've seen how to generate multiplication triples in $GF(2)$, but all of the example applications in prior chapters were in $GF(p)$ rather than $GF(2)$, so this doesn't seem very useful. We'll see additional applications in later chapters where they are useful, but for now it would be great to be able to generate triples in $GF(p)$ instead. This is a more difficult problem that requires a slightly more complex protocol to solve. We'll start by assuming that the parties will generate shares of $a$ and $b$ randomly, then compute $c$ by multiplying. Recall that:

$$
\begin{align*}
c &= (a_1 + a_2)(b_1 + b_2)\\
&= a_1 b_1 + a_2 b_2 + a_1 b_2 + a_2 b_1\\
\end{align*}
$$

The first term involves shares known by P1, and the second one involes shares known by P2. We'll (tentatively) set the shares of $c$ as follows:

$$
\begin{align*}
c_1' &= a_1 b_1\\
c_2' &= a_2 b_2\\
\end{align*}
$$

The parties can compute these shares locally, without communication. However, these are not quite the shares we want: $c_1' + c_2'$ is missing the cross-terms $a_1 b_2 + a_2 b_1$ from the earlier equation.

To solve this problem, we'll use OT to build secret shares of $t = a_1 b_2$ and $u = a_2 b_1$. Specifically, we'll generate the shares:
- $t_1 = - r$ where P1 knows $r$ but doesn't know $a_1 b_2$
- $t_2 = r + a_1 b_2$ where P2 *doesn't* know $r$
- $u_1 = - s$ where P1 knows $s$ but doesn't know $a_2 b_1$
- $u_2 = s + a_2 b_1$ where P2 *doesn't* know $s$

Here, the values $r$ and $s$ act as *masks* to allow sharing the cross-term's value without revealing it. Once we've accomplished this, we can set the two shares of $c$:
- $c_1 = c_1' + t_1 + u_1 = a_1 b_1 - r - s$
- $c_2 = c_2' + t_2 + u_2 = a_2 b_2 + r + a_1 b_2 + s + a_2 b_1$

With this setup, we get that $c_1 + c_2 = a b$, as desired.

To build the secret shares of $t$ and $u$, we'll use a process called *bit decomposition* to convert arithmetic shares into binary digits, and then use 1-out-of-2 OT to deliver a bitwise masked version of the correct share to P2. P2 will be able to sum the bits to obtain their shares of $t$ and $u$. Here's a function `bit_decompose` to turn a field element into an equivalent list of bits. For a field with characteristic $p$, we'll need $\lceil \log_2(p) \rceil$ bits to represent the number.

In [233]:
# We use a small p to make the results easy to read
p = 17
GF = galois.GF(p)

@pychor.local_function
def bit_decompose(x, nbits):
    # Convert the input to an integer
    x = int(x)
    # Return a list of bits corresponding to x
    # the least-significant bit is the first element
    return [(x >> i) & 1 for i in range(nbits)]

# Bit-decompose the field element 15
bit_decompose(GF(15), int(np.ceil(np.log2(p))))

[1, 1, 1, 1, 0]

The following protocol implements the process of generating shares $t_1$ and $t_2 \in GF(p)$ of the cross-term $a_1 b_2$. We denote indexing into a vector as $V[j]$ (instead of $V_j$) to avoid confusion with the subscripts in the names of secret shares.

````{admonition} Protocol: Secret Share a Cross-Term
:class: note

The parties P1 and P2 follow the following steps:
1. P2 bit-decomposes $b_2$ into a vector $B_2 \in \{0, 1\}^n$ where $n = \lceil \log_2(p) \rceil$.
2. P1 generates a random length-$n$ vector of field elements $R \in GF(p)^n$ to mask the OT results.
3. For $j \in 0 \dots n-1$, P1 and P2 run OT, with the secret inputs $R[j]$ and $R[j] + a_1 * 2^j$ and the selection bit $B_2[j]$. P2 receives the vector of results $O \in GF(p)^n$.
4. P1 computes $t_1 = -\sum R \mod p$ and P2 computes $t_2 = \sum O \mod p$.

At the end of the protocol, the parties hold additive shares of $a_1 b_2$.
````

In [306]:
def protocol_crossterm(a1, b2):
    # Function to generate the secret inputs for OT
    @pychor.local_function
    def table(j, a1, rj):
        return [rj, (rj + int(a1) * 2**j) % p]

    # Step 1: P2 bit decomposes b2
    num_bits = int(np.ceil(np.log2(p)))
    b2_bits = bit_decompose(b2, num_bits).unlist(num_bits)

    # Step 2: P1 generates a random vector r to mask a1*b2
    r_vec = [p1.constant(np.random.randint(0, p)) for _ in range(num_bits)]
    r = sum(r_vec) % p

    # Step 3: P1 and P2 run OT to get rj+a1*b2[j]*2^j for each j
    ot_results = [protocol_ot(sender=p1, receiver=p2, 
                              inputs=table(j, a1, r_vec[j]), 
                              selection=b2_bits[j], 
                              n=2)
                  for j in range(num_bits)]

    # Step 4: Compute shares
    # P1's share is -r
    # P2's share is sum(rj+a1*b2[j]*2^j) = r + a1*b2
    t1 = pychor.locally(GF, (-r) % p)
    t2 = pychor.locally(GF, sum(ot_results) % p)

    return t1, t2

In [307]:
with pychor.LocalBackend():
    a1 = p1.constant(GF(1))
    b2 = p2.constant(GF(4))
    print('Cross term shares:', protocol_crossterm(a1, b2))

Cross term shares: (11@{p1}, 10@{p2})


Now we're able to define a protocol for generating an arithmetic multiplication triple:

````{admonition} Protocol: Generate Arithmetic Multiplication Triple
:class: note

The parties P1 and P2 follow the following steps:
1. P1 picks $a_1$, $b_1 \in GF(p)$ randomly
2. P2 picks $a_2$, $b_2 \in GF(p)$ randomly
3. P1 and P2 run `protocol_crossterm` with the inputs $a_1$, $b_2$ to obtain shares of $t = a_1 b_2$. P1 gets $t_1$ and P2 gets $t_2$.
4. P1 and P2 run `protocol_crossterm` with the inputs $b_1$, $a_2$ to obtain shares of $u = a_2 b_1$. P1 gets $u_1$ and P2 gets $u_2$.
5. P1 computes the share $c_1 = a_1 b_1 + t_1 + u_1$
6. P2 computes the share $c_2 = a_2 b_2 + t_2 + u_2$

At the end of the protocol, P1 and P2 hold secret shares of $a$, $b$, and $c$, such that $a$ and $b$ are random and unknown to either party, and $a*b = c$.
````

In [310]:
def protocol_gen_arithmetic_mult_triple():
    # Step 1: Pick P1's shares
    a1 = p1.constant(GF.Random())
    b1 = p1.constant(GF.Random())

    # Step 2: Pick P2's shares
    a2 = p2.constant(GF.Random())
    b2 = p2.constant(GF.Random())

    # Step 3: Obtain shares of a1 * b2
    t1, t2 = protocol_crossterm(a1, b2)

    # Step 4: Obtain shares of b1 * a2
    u1, u2 = protocol_crossterm(b1, a2)

    # Step 5 & 6: Compute shares
    c1 = a1 * b1 + t1 + u1
    c2 = a2 * b2 + t2 + u2

    return (a1, a2), (b1, b2), (c1, c2)

In [311]:
with pychor.LocalBackend():
    print('Multiplication triple:', protocol_gen_arithmetic_mult_triple())

Multiplication triple: ((14@{p1}, 2@{p2}), (7@{p1}, 1@{p2}), (16@{p1}, 10@{p2}))


Note that this protocol requires two uses of the `protocol_crossterm` protocol, and each one of those requires $\lceil \log_2(p) \rceil$ uses of OT. Each OT requires two public-key encryptions, which is often the most expensive part of one of these protocols. This means that generating multiplication triples in $GF(p)$ is expensive!

````{admonition} Further reading: OT extension
:class: seealso

The idea of *Oblivious Transfer extension*, introduced in {cite}`ishai2003extending`, speeds up protocols that make use of OT in batches (e.g. for generating multiplication triples) by "extending" a small number of "base OTs" (e.g. 128) to a much larger number of usable OTs (e.g. thousands). Techniques for OT extension are surveyed in **Section 3.7.2 of [Pragmatic MPC](https://securecomputation.org/)**.
````